In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cartopy as cy
import cartopy.crs as ccrs
import matplotlib.path as mpath
import s3fs

In [2]:
#data_dir = '/home/jovyan/Tjaernoe2022-group5/data'
scenarios = ['ssp245','ssp585']

In [6]:
def circle_for_polar_map(axes):
    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)
    axes.set_boundary(circle, transform=axes.transAxes)


def count_ARs(ds, lat_cut):
    '''
    Counts number of AR at each time step. Returns dataset with added coordinate ar_counts_[lat_cut].
    '''
    if lat_cut<0:
        pole_ds = ds.sel(lat= slice(-90, lat_cut))
    else:
        pole_ds = ds.sel(lat= slice(lat_cut, 90))

    ar_counts = np.zeros(len(pole_ds.time))
    for i,ts in enumerate(pole_ds.time):
        ll = xr.plot.contour(pole_ds.sel(time=ts).ivt, levels=[0.0,1.0])
        plt.close()
        if len(ll.collections)>1: #You can remove this and next line if you have run it a few times without getting the printout :))
            print('julia was wrong about something, tell her to fix it'+ts) 
        nr_ar = len(ll.collections[0].get_paths())
        ar_counts[i] = nr_ar
    ds[f'ar_counts_{lat_cut}']= ar_counts
    return ds

In [29]:
ar_dict = {}
for s in scenarios:
    near_ar = xr.open_dataset(f'{s}_20352050_AR_detection.nc')
    future_ar = xr.open_dataset(f'{s}_20852100_AR_detection.nc')
    ar_dict[s] = {'near': near_ar, 'future' : future_ar}
    

In [7]:
ar_dict['ssp245']['near']= count_ARs(ar_dict['ssp245']['near'], 60)
ar_dict['ssp245']['near']= count_ARs(ar_dict['ssp245']['near'], -60)

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/plot/plot.py:1417: UserWarning: No contour levels were found within the data range.
  primitive = ax.contour(x, y, z, **kwargs)


In [8]:
ar_dict['ssp245']['near']

<xarray.Dataset>
Dimensions:        (lat: 96, lon: 144, time: 5476, ar_counts_60: 5476,
                    ar_counts_-60: 5476)
Coordinates:
  * lat            (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon            (lon) float64 0.0 2.5 5.0 7.5 ... 350.0 352.5 355.0 357.5
  * time           (time) object 2035-01-01 12:00:00 ... 2050-01-01 12:00:00
  * ar_counts_60   (ar_counts_60) float64 0.0 0.0 0.0 0.0 ... 1.0 2.0 0.0 0.0
  * ar_counts_-60  (ar_counts_-60) float64 0.0 0.0 0.0 1.0 ... 0.0 0.0 0.0 1.0
Data variables:
    ivt            (lat, time, lon) bool ...

In [30]:
polar_cut = 60
for s in ar_dict.keys():
    for period in ['near','future']:
        ar_dict[s][period]= count_ARs(ar_dict[s][period], polar_cut)
        ar_dict[s][period]= count_ARs(ar_dict[s][period], -polar_cut)
        ar_dict[s][period].close()
        print(f'{s} {period} done')

/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/plot/plot.py:1417: UserWarning: No contour levels were found within the data range.
  primitive = ax.contour(x, y, z, **kwargs)


ssp245 near done
ssp245 future done
ssp585 near done
ssp585 future done


In [34]:
ar_dict['ssp245']['near']['ar_counts_60'].sum()/15

<xarray.DataArray 'ar_counts_60' ()>
array(359.73333333)

In [ ]:
s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", 
                       secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0", 
                       client_kwargs=dict(endpoint_url="https://rgw.met.no"))
s3.ls('escience2022/Remy/‘)

In [ ]:
s3path = "s3://escience2022/Remy/od550aer_AERday_NorESM2-LM_historical_r1i1p1f1_gn_20000101-20091231.nc”

esacci = xr.open_dataset(s3.open(s3path))